In [1]:
import numpy as np
import awkward as ak
import warnings
import matplotlib.pyplot as plt
import hist
import math
import os
import json
import gc
import dask_awkward as dak

In [2]:
path = '/scratch365/cmoore24/training/hgg/batch2024/ml_results_checking'
with open('../event_totals.json', 'r') as f:
    totals = json.load(f)

In [3]:
with open(f'{path}/my_xsecs.json', 'r') as f:
    xsecs = json.load(f)

In [4]:
xsecs2 = {}
for i in xsecs:
    if type(xsecs[i]) == dict:
        for j in xsecs[i]:
            xsecs2[j] = xsecs[i][j]
    else:
        xsecs2[i] = xsecs[i]
xsecs = xsecs2

In [5]:
totals2 = {}
for i in totals:
    if type(totals[i]) == dict:
        for j in totals[i]:
            totals2[f'{i}_{j}'] = totals[i][j]
    else:
        totals2[i] = totals[i]
totals = totals2

In [6]:
def read_files(input_path):
    if ('.parquet' in os.listdir(input_path)[0]):
        output = ak.from_parquet(f'{input_path}/*', columns=columns)
    else:
        output = {}
        for i in os.listdir(input_path):
            if ('flat400' not in i):
                output[i] = ak.from_parquet(f'{input_path}/{i}/*', columns=columns)
            else:
                continue
    return output

In [7]:
totals['ww'] = totals['diboson_ww']
totals['wz'] = totals['diboson_wz']
totals['zz'] = totals['diboson_zz']

In [8]:
region = 'nolepton'
path = '/project01/ndcms/cmoore24/skims/full_skims'

In [9]:
def firsts(mc):
    for i in mc:
        if type(mc[i]) == dict:
            for j in mc[i]:
                for k in mc[i][j].fields:
                    if 'event' in k:
                        continue
                    else:
                        try:
                            mc[i][j][k] = ak.firsts(mc[i][j][k])
                        except:
                            continue
        else:
            for j in mc[i].fields:
                if 'event' in j:
                    continue
                else:
                    try:
                        mc[i][j] = ak.firsts(mc[i][j])
                    except:
                        continue
    return mc

In [10]:
ecf_list = dak.from_parquet('/project01/ndcms/cmoore24/skims/full_skims/nolepton/mc/hgg/*').groomed_ecfs.fields

In [11]:
upper = 1200
lower = 500
IL = 44.99

In [17]:
def ecf_hist(dataset, ecf_min, ecf_max, var):
    make_hist = hist.Hist.new.Reg(40, ecf_min, ecf_max, name='ECF', label='MC ECF').Weight()
    make_hist.fill(ECF=dataset.groomed_ecfs[var])
    return make_hist

In [15]:
# ecf_ks = {}
# with open('groomed_ks.json', 'w') as f:
#     json.dump(ecf_ks, f)

In [18]:
for k in range(0, len(ecf_list)):
    columns=['goodjets.msoftdrop', 'goodjets.pt', ("groomed_ecfs", f'{ecf_list[k]}')]
    mc = read_files(f'{path}/{region}/mc')
    mc['ww'] = mc['diboson_ww']
    mc['wz'] = mc['diboson_wz']
    mc['zz'] = mc['diboson_zz']
    del(mc['diboson_ww'])
    del(mc['diboson_wz'])
    del(mc['diboson_zz'])
    mc = firsts(mc)
    data = read_files(f'{path}/{region}/data')
    data = firsts(data)

    for i in xsecs:
        if type(mc[i]) == dict:
            for j in mc[i]:
                mask = ((mc[i][j].goodjets.pt >= lower) & (mc[i][j].goodjets.pt <= upper))
                mc[i][j] = mc[i][j][mask]
        else:
            mask = ((mc[i].goodjets.pt >= lower) & (mc[i].goodjets.pt <= upper))
            mc[i] = mc[i][mask]

    for i in data:
        if type(data[i]) == dict:
            for j in data[i]:
                mask = ((data[i][j].goodjets.pt >= lower) & (data[i][j].goodjets.pt <= upper))
                data[i][j] = data[i][j][mask]
        else:
            mask = ((data[i].goodjets.pt >= lower) & (data[i].goodjets.pt <= upper))
            data[i] = data[i][mask]

    data_s = {}
    for i in data:
        if "Jet" in i:
            data_s[i] = data[i]  
    data_arr = ak.concatenate([data[i] for i in data_s])

    var = ecf_list[k]

    ecf_max = ak.max(data_arr.groomed_ecfs[var])
    ecf_min = ak.min(data_arr.groomed_ecfs[var])

    data_hist = hist.Hist.new.Reg(40, ecf_min, ecf_max, name='ECF', label='Data ECF').Weight()
    data_hist.fill(ECF=data_arr.groomed_ecfs[var])

    mc2 = {}
    for i in xsecs:
        if type(mc[i]) == dict:
            for j in mc[i]:
                mc2[j] = mc[i][j]
        else:
            mc2[i] = mc[i]
    mc = mc2    

    hists = {}
    for i in mc:
        if type(mc[i]) == dict:
            hists[i] = {}
            for j in mc[i]:
                hists[i][j] = msd_hist(mc[i][j])
        else:
            hists[i] = ecf_hist(mc[i], ecf_min, ecf_max, var)

    scaleHgg = ((IL*(xsecs['hgg']*1000)*0.0817)/(totals['hgg']))
    hists['hgg'].view(flow=True)[:] *= scaleHgg
    
    scaleHbb = ((IL*(xsecs['hbb']*1000)*0.581)/(totals['hbb']))
    hists['hbb'].view(flow=True)[:] *= scaleHbb

    for i in mc:
        if (i == 'hgg') or (i == 'hbb'):
            continue
        else:
            scale = ((IL*(xsecs[i]*1000))/(totals[i]))
            hists[i].view(flow=True)[:] *= scale

    mc_hist = sum(hists[i] for i in hists)

    mc_values, mc_bins = mc_hist.to_numpy()
    data_values, data_bins = data_hist.to_numpy()
    mc_density = mc_values / mc_values.sum()
    data_density = data_values / data_values.sum()
    mc_cdf = np.cumsum(mc_density)
    data_cdf = np.cumsum(data_density)
    ks_statistic = np.max(np.abs(mc_cdf - data_cdf))
    adjusted = -math.log10(ks_statistic)

    with open('groomed_ks.json','r') as f:
        ecf_ks = json.load(f)

    ecf_ks[var] = adjusted

    with open('groomed_ks.json','w') as f:
        json.dump(ecf_ks, f)

    print(f'{var} is done')
    gc.collect()
    if k % 10 == 0:
        print(k)

1e2^0.5 is done
0
1e2^1.0 is done
1e2^1.5 is done
1e2^2.0 is done
1e2^2.5 is done
1e2^3.0 is done
1e2^3.5 is done
1e2^4.0 is done
1e3^0.5 is done
1e3^1.0 is done
1e3^1.5 is done
10
1e3^2.0 is done
1e3^2.5 is done
1e3^3.0 is done
1e3^3.5 is done
1e3^4.0 is done
2e3^0.5 is done
2e3^1.0 is done
2e3^1.5 is done
2e3^2.0 is done
2e3^2.5 is done
20
2e3^3.0 is done
2e3^3.5 is done
2e3^4.0 is done
3e3^0.5 is done
3e3^1.0 is done
3e3^1.5 is done
3e3^2.0 is done
3e3^2.5 is done
3e3^3.0 is done
3e3^3.5 is done
30
3e3^4.0 is done
1e4^0.5 is done
1e4^1.0 is done
1e4^1.5 is done
1e4^2.0 is done
1e4^2.5 is done
1e4^3.0 is done
1e4^3.5 is done
1e4^4.0 is done
2e4^0.5 is done
40
2e4^1.0 is done
2e4^1.5 is done
2e4^2.0 is done
2e4^2.5 is done
2e4^3.0 is done
2e4^3.5 is done
2e4^4.0 is done
3e4^0.5 is done
3e4^1.0 is done
3e4^1.5 is done
50
3e4^2.0 is done
3e4^2.5 is done
3e4^3.0 is done
3e4^3.5 is done
3e4^4.0 is done
4e4^0.5 is done
4e4^1.0 is done
4e4^1.5 is done
4e4^2.0 is done
4e4^2.5 is done
60
4e4^